# ES → BBF Salesforce Service_Charge Migration

This notebook migrates OrderItem records from ES Salesforce to Service_Charge__c in BBF Salesforce.

## Prerequisites
- **Service__c migration must be completed first** (Orders have BBF_New_Id__c populated)
- ES OrderItem records must have BBF_New_Id__c field created

## Object Mapping
- **ES Source:** OrderItem (Salesforce Standard Object with custom fields)
- **BBF Target:** Service_Charge__c (Custom Object)

## Process Overview
1. Connect to both ES (source) and BBF (target) Salesforce orgs
2. Query OrderItem from ES where:
   - Parent Order has BBF_New_Id__c populated (Service already migrated)
   - Parent Order Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
   - Parent Order Project_Group__c NOT LIKE '%PA MARKET DECOM%'
   - Parent Order Service_Order_Record_Type__c = 'Service Order Agreement'
   - OrderItem does NOT have BBF_New_Id__c populated (not yet migrated)
3. **DUPLICATE PREVENTION**: Check BBF for existing ES_Legacy_ID__c to filter out already-migrated records
4. Transform ES OrderItem for BBF Service_Charge__c schema:
   - Map Order.BBF_New_Id__c → BBF Service ID (MASTER-DETAIL REQUIRED)
   - Map Product2 → Product_Simple__c picklist
   - Map charge type fields → Service_Type_Charge__c picklist
   - Set all Boolean fields to False (11 required fields)
   - Add ES_Legacy_ID__c = ES OrderItem.Id (for tracking)
5. Insert Service_Charge__c to BBF Salesforce
6. Update ES OrderItem with BBF_New_Id__c = BBF Service_Charge.Id
7. Create ID mapping: ES OrderItem ID → BBF Service_Charge ID
8. Output results to Excel with color-coded status

## Field Tracking Strategy
**In BBF Service_Charge__c:** `ES_Legacy_ID__c` stores original ES OrderItem ID
- Text(18), External ID, Unique (if available)

**In ES OrderItem:**
- `BBF_New_Id__c` stores new BBF Service_Charge ID after migration (Text 18)

## Safety
- `TEST_MODE = True` by default (limits to 10 Service Charges)
- Only migrates OrderItems where parent Order/Service is already migrated
- Skips OrderItems already migrated (BBF_New_Id__c populated in ES)
- **DUPLICATE PREVENTION**: Checks BBF ES_Legacy_ID__c before insert to prevent duplicates on rerun
  - This handles the case where BBF insert succeeded but ES update failed (row lock)
- Skips OrderItems where parent Service not yet migrated
- Comprehensive error handling for master-detail relationship requirements

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from datetime import datetime
import os

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Set-up successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Set-up successful


In [2]:
# === CONFIGURATION ===

# ES UAT Credentials
ES_USERNAME = "sfdcapi@everstream.net.uat"
ES_PASSWORD = "ZXasqw1234!@#$"
ES_TOKEN = "X0ation2CNmK5C0pV94M6vFYS"
ES_DOMAIN = "test"

# # ES (Source) Credentials - Production
# ES_USERNAME = "sfdcapi@everstream.net"
# ES_PASSWORD = "pV4CAxns8DQtJsBq!"
# ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
# ES_DOMAIN = "login"  # or 'test' for sandbox

# BBF (Target) Credentials
BBF_USERNAME = "vlettau@everstream.net"
BBF_PASSWORD = "MNlkpo0987)(*&"
BBF_TOKEN = "I4xmQLmm03cXl1O9qI2Z3XAAX"
BBF_DOMAIN = "test"  # or 'login' for production

# Migration Options
TEST_MODE = False  # ⚠️ Set to False to migrate ALL Service Charges
TEST_LIMIT = 10  # Only used when TEST_MODE = True

# 👤 Service Charge Owner - Set all migrated Service Charges to this user
OWNER_ID = (
    "005Ea00000ZOGFZIA5"  # Same as Account/Contact/BAN/Location/Service migration
)

# Output Configuration
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f"es_bbf_service_charge_migration_{timestamp}.xlsx"

print("📋 Configuration loaded")
print(f"   TEST_MODE: {TEST_MODE}")
print(f"   Owner ID: {OWNER_ID}")
print(f"   Output: {output_file}")
print("\n⚠️  Note: Bulk API automatically handles batching (200 records/batch)")

📋 Configuration loaded
   TEST_MODE: False
   Owner ID: 005Ea00000ZOGFZIA5
   Output: es_bbf_service_charge_migration_20260123_161715.xlsx

⚠️  Note: Bulk API automatically handles batching (200 records/batch)


In [3]:
# === CONNECT TO SALESFORCE ORGS ===

print("=" * 80)
print("CONNECTING TO SALESFORCE ORGS")
print("=" * 80)

# Connect to ES (source)
print("\n📌 Connecting to ES (source)...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

# Connect to BBF (target)
print("\n📌 Connecting to BBF (target)...")
bbf_sf = Salesforce(
    username=BBF_USERNAME,
    password=BBF_PASSWORD,
    security_token=BBF_TOKEN,
    domain=BBF_DOMAIN,
)
print(f"✅ Connected to BBF: {bbf_sf.sf_instance}")

CONNECTING TO SALESFORCE ORGS

📌 Connecting to ES (source)...
✅ Connected to ES: everstream--uat.sandbox.my.salesforce.com

📌 Connecting to BBF (target)...
✅ Connected to BBF: bluebirdnetwork--full.sandbox.my.salesforce.com


In [ ]:
# === QUERY ES ORDERITEM (Service Charges) ===
# Day 1: MINIMAL FIELDS ONLY
# BBF Service_Charge__c: Name is AUTONUMBER, no OwnerId
# Required: Service__c (master-detail), Product_Simple__c, Service_Type_Charge__c
# Product mapping not ready yet - using placeholder values for now

print("\n" + "=" * 80)
print("QUERYING ES ORDERITEM (Service Charges) - MINIMAL FIELDS")
print("=" * 80)

# MINIMAL QUERY - Only fields needed for Day 1:
# - Id: for ES_Legacy_ID__c tracking
# - Order.BBF_New_Id__c: for master-detail to Service__c
query = """SELECT Id, Order.BBF_New_Id__c
FROM OrderItem
WHERE Order.BBF_New_Id__c != null
AND Order.BBF_New_Id__c != ''
AND Order.Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
AND (Order.Project_Group__c = null OR (NOT Order.Project_Group__c LIKE '%PA MARKET DECOM%'))
AND Order.Service_Order_Record_Type__c = 'Service Order Agreement'
AND (BBF_New_Id__c = null OR BBF_New_Id__c = '')"""

if TEST_MODE:
    query += f" LIMIT {TEST_LIMIT}"

print(f"Query:\n{query}")
print("\nExecuting query...")

result = es_sf.query_all(query)
es_orderitems_raw = result["records"]

print(f"✅ Found {len(es_orderitems_raw)} OrderItem records from ES query")

# =============================================================================
# DUPLICATE PREVENTION: Check BBF for already-migrated records
# If record exists in BBF, get its Id to update ES BBF_New_Id__c
# =============================================================================
print("\n" + "-" * 80)
print("DUPLICATE PREVENTION: Checking BBF for existing ES_Legacy_ID__c...")
print("-" * 80)

bbf_existing_query = """
SELECT Id, ES_Legacy_ID__c 
FROM Service_Charge__c 
WHERE ES_Legacy_ID__c != null
"""
bbf_existing_result = bbf_sf.query_all(bbf_existing_query)

# Build lookup: ES_Legacy_ID__c -> BBF Id
existing_bbf_lookup = {r['ES_Legacy_ID__c']: r['Id'] for r in bbf_existing_result['records']}

print(f"   Found {len(existing_bbf_lookup)} Service_Charge__c records already in BBF")

# Separate: records to migrate vs records that need ES BBF_New_Id__c sync
es_orderitems = []
es_needs_sync = []  # Records that exist in BBF but ES.BBF_New_Id__c is null

for oi in es_orderitems_raw:
    if oi['Id'] in existing_bbf_lookup:
        # Already in BBF - need to sync ES.BBF_New_Id__c
        es_needs_sync.append({
            'es_id': oi['Id'],
            'bbf_id': existing_bbf_lookup[oi['Id']]
        })
    else:
        # Not in BBF - need to migrate
        es_orderitems.append(oi)

print(f"   Records to migrate (not in BBF): {len(es_orderitems)}")
print(f"   Records to sync (in BBF, ES.BBF_New_Id__c missing): {len(es_needs_sync)}")

# Sync ES.BBF_New_Id__c for records that already exist in BBF
if len(es_needs_sync) > 0:
    print(f"\n📌 Syncing {len(es_needs_sync)} ES OrderItem BBF_New_Id__c values...")
    
    sync_updates = [{'Id': item['es_id'], 'BBF_New_Id__c': item['bbf_id']} for item in es_needs_sync]
    
    try:
        sync_results = es_sf.bulk.OrderItem.update(sync_updates)
        sync_success = sum(1 for r in sync_results if r['success'])
        sync_failed = sum(1 for r in sync_results if not r['success'])
        
        print(f"   ✅ Synced: {sync_success}")
        print(f"   ❌ Failed to sync: {sync_failed}")
        
        if sync_failed > 0:
            print("   First 5 sync failures:")
            fail_count = 0
            for i, r in enumerate(sync_results):
                if not r['success'] and fail_count < 5:
                    print(f"     - {sync_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1
    except Exception as e:
        print(f"   ❌ Error syncing: {e}")

print(f"\n✅ {len(es_orderitems)} OrderItem records to migrate (after duplicate check)")

if len(es_orderitems) > 0:
    sample = es_orderitems[0]
    print(f"\nSample OrderItem:")
    print(f"  ID:              {sample['Id']}")
    print(f"  BBF Service ID:  {sample.get('Order', {}).get('BBF_New_Id__c', 'N/A')}")
else:
    print("\n✅ All OrderItem records have been migrated!")

In [5]:
# === TRANSFORM FOR BBF SERVICE_CHARGE__c ===
# Day 1 Migration: REQUIRED FIELDS ONLY
# - Name: AUTONUMBER (don't set)
# - No OwnerId field on Service_Charge__c
# - Product_Simple__c and Service_Type_Charge__c: PLACEHOLDER VALUES
#   (Business mapping not ready - will be updated in Day 2 enrichment)

print("\n" + "=" * 80)
print("TRANSFORMING ORDERITEM FOR BBF SERVICE_CHARGE__c (REQUIRED FIELDS ONLY)")
print("=" * 80)

# ⚠️ PLACEHOLDER VALUES - Business product mapping not ready yet
# These will be updated via enrichment process once mapping is provided
PLACEHOLDER_PRODUCT = "ANNUAL"  # Temporary placeholder
PLACEHOLDER_SERVICE_TYPE = "Power"  # Temporary placeholder

print(f"⚠️  Using PLACEHOLDER values (to be enriched later):")
print(f"   Product_Simple__c = '{PLACEHOLDER_PRODUCT}'")
print(f"   Service_Type_Charge__c = '{PLACEHOLDER_SERVICE_TYPE}'")

bbf_service_charges = []
skipped_no_bbf_service = []

for es_orderitem in es_orderitems:
    bbf_service_id = None
    if es_orderitem.get("Order") and es_orderitem["Order"].get("BBF_New_Id__c"):
        bbf_service_id = es_orderitem["Order"]["BBF_New_Id__c"]

    if not bbf_service_id:
        skipped_no_bbf_service.append(
            {
                "es_id": es_orderitem["Id"],
                "reason": "No BBF Service ID - Master-Detail required",
            }
        )
        continue

    # =========================================================================
    # BBF Service_Charge__c - REQUIRED FIELDS ONLY
    # Name = Autonumber (don't set)
    # No OwnerId on Service_Charge__c
    # Booleans default to False (don't need to set)
    # =========================================================================
    bbf_service_charge = {
        # 🔴 REQUIRED: Master-Detail to Service__c
        "Service__c": bbf_service_id,
        # 🔴 REQUIRED: Picklists (PLACEHOLDER - to be enriched)
        "Product_Simple__c": PLACEHOLDER_PRODUCT,
        "Service_Type_Charge__c": PLACEHOLDER_SERVICE_TYPE,
        # 🔗 Tracking
        "ES_Legacy_ID__c": es_orderitem["Id"],
    }

    bbf_service_charges.append(bbf_service_charge)

print(f"\n✅ Transformed {len(bbf_service_charges)} Service Charges")
print(f"\n   REQUIRED FIELDS SET:")
print(f"   - Service__c (Master-Detail to Service)")
print(f"   - Product_Simple__c = '{PLACEHOLDER_PRODUCT}' (PLACEHOLDER)")
print(f"   - Service_Type_Charge__c = '{PLACEHOLDER_SERVICE_TYPE}' (PLACEHOLDER)")
print(f"   - ES_Legacy_ID__c (tracking)")
print(f"\n   Note: Name is Autonumber, no OwnerId, booleans default False")

if len(skipped_no_bbf_service) > 0:
    print(f"\n⚠️  Skipped {len(skipped_no_bbf_service)} (no BBF Service ID)")


TRANSFORMING ORDERITEM FOR BBF SERVICE_CHARGE__c (REQUIRED FIELDS ONLY)
⚠️  Using PLACEHOLDER values (to be enriched later):
   Product_Simple__c = 'ANNUAL'
   Service_Type_Charge__c = 'Power'

✅ Transformed 17709 Service Charges

   REQUIRED FIELDS SET:
   - Service__c (Master-Detail to Service)
   - Product_Simple__c = 'ANNUAL' (PLACEHOLDER)
   - Service_Type_Charge__c = 'Power' (PLACEHOLDER)
   - ES_Legacy_ID__c (tracking)

   Note: Name is Autonumber, no OwnerId, booleans default False


In [6]:
# === INSERT TO BBF ===

print("\n" + "=" * 80)
print("INSERTING SERVICE CHARGES TO BBF")
print("=" * 80)

if len(bbf_service_charges) == 0:
    print("⚠️  No Service Charges to insert")
    successful_inserts = []
    failed_inserts = []
else:
    print(f"Inserting {len(bbf_service_charges)} Service Charges using bulk API...")
    print("(Bulk API automatically batches in 200-record chunks)\n")

    try:
        results = bbf_sf.bulk.Service_Charge__c.insert(bbf_service_charges)

        successful_inserts = []
        failed_inserts = []

        for i, result in enumerate(results):
            if result["success"]:
                successful_inserts.append(
                    {
                        "es_id": bbf_service_charges[i]["ES_Legacy_ID__c"],
                        "bbf_id": result["id"],
                        "bbf_service_id": bbf_service_charges[i]["Service__c"],
                        "product": bbf_service_charges[i]["Product_Simple__c"],
                        "service_type": bbf_service_charges[i][
                            "Service_Type_Charge__c"
                        ],
                    }
                )
            else:
                failed_inserts.append(
                    {
                        "es_id": bbf_service_charges[i]["ES_Legacy_ID__c"],
                        "errors": result["errors"],
                        "bbf_service_id": bbf_service_charges[i]["Service__c"],
                    }
                )

        print(f"✅ Successfully inserted: {len(successful_inserts)} Service Charges")
        print(f"❌ Failed to insert: {len(failed_inserts)} Service Charges")

        if len(failed_inserts) > 0:
            print(f"\nFailed Service Charges (first 5):")
            for item in failed_inserts[:5]:
                print(f"  - ES ID: {item['es_id']}")
                print(f"    Errors: {item['errors']}")

        if len(successful_inserts) > 0:
            print(f"\nSample successful insert:")
            sample = successful_inserts[0]
            print(f"  ES OrderItem ID:          {sample['es_id']}")
            print(f"  BBF Service_Charge ID:    {sample['bbf_id']}")
            print(f"  BBF Service ID:           {sample['bbf_service_id']}")
            print(f"  Product (placeholder):    {sample['product']}")
            print(f"  Service Type (placeholder): {sample['service_type']}")

    except Exception as e:
        print(f"❌ Error during bulk insert: {e}")
        successful_inserts = []
        failed_inserts = []


INSERTING SERVICE CHARGES TO BBF
Inserting 17709 Service Charges using bulk API...
(Bulk API automatically batches in 200-record chunks)

✅ Successfully inserted: 17704 Service Charges
❌ Failed to insert: 5 Service Charges

Failed Service Charges (first 5):
  - ES ID: 8023g000004tOVIAA2
    Errors: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - ES ID: 8023g000004tS2pAAE
    Errors: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - ES ID: 802Rn00000YKxjWIAT
    Errors: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - ES ID: 802Rn00000YhLiLIAV
    Errors: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - ES ID: 802Rn00000Z8HkQIAV
    Errors: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain 

In [7]:
# === UPDATE ES WITH BBF IDS ===

print("\n" + "=" * 80)
print("UPDATING ES WITH BBF IDS")
print("=" * 80)

if len(successful_inserts) == 0:
    print("⚠️  No Service Charges to update in ES")
    es_update_results = []
else:
    # Build update records for ES - set BBF_New_Id__c only
    es_updates = [
        {"Id": item["es_id"], "BBF_New_Id__c": item["bbf_id"]}
        for item in successful_inserts
    ]

    print(f"Updating {len(es_updates)} OrderItem records in ES...")
    print("   - Setting BBF_New_Id__c = BBF Service_Charge ID")

    try:
        es_update_results = es_sf.bulk.OrderItem.update(es_updates)

        success_count = sum(1 for r in es_update_results if r["success"])
        error_count = sum(1 for r in es_update_results if not r["success"])

        print(f"\n✅ Successfully updated: {success_count} OrderItem records in ES")
        print(f"❌ Failed to update: {error_count} OrderItem records in ES")

        if error_count > 0:
            print("\nFirst 10 update failures:")
            fail_count = 0
            for i, r in enumerate(es_update_results):
                if not r["success"] and fail_count < 10:
                    print(f"  - {es_updates[i]['Id']}: {r['errors']}")
                    fail_count += 1

    except Exception as e:
        print(f"❌ Error during ES update: {e}")
        es_update_results = []


UPDATING ES WITH BBF IDS
Updating 17704 OrderItem records in ES...
   - Setting BBF_New_Id__c = BBF Service_Charge ID

✅ Successfully updated: 15206 OrderItem records in ES
❌ Failed to update: 2498 OrderItem records in ES

First 10 update failures:
  - 8023g0000000KNwAAM: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - 8023g0000000KpJAAU: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - 8023g0000000KfuAAE: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - 8023g0000000KfvAAE: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - 8023g0000000KfwAAE: [{'statusCode': 'UNABLE_TO_LOCK_ROW', 'message': 'unable to obtain exclusive access to this record', 'fields': []}]
  - 8023g0000000KfxAAE: [{'statusCode': 'UNABLE_TO

In [8]:
# === CREATE EXCEL OUTPUT ===

print("\n" + "=" * 80)
print("CREATING EXCEL OUTPUT")
print("=" * 80)

wb = Workbook()
ws1 = wb.active
ws1.title = "Migration Results"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)

status_colors = {
    "Success": "C6EFCE",
    "Failed": "FFC7CE",
    "Skipped": "FFEB9C",
}

# --- SHEET 1: Migration Results ---
headers1 = [
    "ES OrderItem ID",
    "BBF Service_Charge ID",
    "BBF Service ID",
    "Product (placeholder)",
    "Service Type (placeholder)",
    "Status",
    "Error",
]
ws1.append(headers1)

for col, header in enumerate(headers1, 1):
    cell = ws1.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment
    cell.border = thin_border

# Combine all results
all_results = []
for item in successful_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": item["bbf_id"],
            "BBF_Service_ID": item["bbf_service_id"],
            "Product": item["product"],
            "Service_Type": item["service_type"],
            "Status": "Success",
            "Error": "",
        }
    )
for item in failed_inserts:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "BBF_Service_ID": item["bbf_service_id"],
            "Product": "",
            "Service_Type": "",
            "Status": "Failed",
            "Error": str(item["errors"]),
        }
    )
for item in skipped_no_bbf_service:
    all_results.append(
        {
            "ES_ID": item["es_id"],
            "BBF_ID": "",
            "BBF_Service_ID": "",
            "Product": "",
            "Service_Type": "",
            "Status": "Skipped",
            "Error": item["reason"],
        }
    )

for row_idx, r in enumerate(all_results, 2):
    ws1.append(
        [
            r["ES_ID"],
            r["BBF_ID"],
            r["BBF_Service_ID"],
            r["Product"],
            r["Service_Type"],
            r["Status"],
            r["Error"],
        ]
    )
    fill_color = status_colors.get(r["Status"], "FFFFFF")
    for col in range(1, len(headers1) + 1):
        cell = ws1.cell(row=row_idx, column=col)
        cell.fill = PatternFill("solid", fgColor=fill_color)
        cell.border = thin_border

for col in ws1.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws1.column_dimensions[col[0].column_letter].width = min(max_length + 2, 60)

ws1.freeze_panes = "A2"

# --- SHEET 2: Summary ---
ws2 = wb.create_sheet("Summary")
ws2.append(["ES → BBF Service_Charge Migration Summary"])
ws2["A1"].font = Font(bold=True, size=14)
ws2.append([])
ws2.append(["Run Type:", "TEST MODE" if TEST_MODE else "FULL MIGRATION"])
ws2.append(["Timestamp:", datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
ws2.append([])
ws2.append(["⚠️ PLACEHOLDER VALUES USED:"])
ws2.append(["Product_Simple__c:", PLACEHOLDER_PRODUCT])
ws2.append(["Service_Type_Charge__c:", PLACEHOLDER_SERVICE_TYPE])
ws2.append([])
ws2.append(["Metric", "Count"])
ws2["A10"].font = Font(bold=True)
ws2["B10"].font = Font(bold=True)
ws2.append(["Total Service Charges Processed", len(all_results)])
ws2.append(["Successful Inserts", len(successful_inserts)])
ws2.append(["Failed Inserts", len(failed_inserts)])
ws2.append(["Skipped (No BBF Service)", len(skipped_no_bbf_service)])
ws2.append(
    [
        "Success Rate",
        (
            f"{len(successful_inserts)/len(all_results)*100:.1f}%"
            if len(all_results) > 0
            else "0%"
        ),
    ]
)

# --- SHEET 3: ID Mapping ---
ws3 = wb.create_sheet("ID Mapping")
headers3 = ["ES OrderItem ID", "BBF Service_Charge ID", "BBF Service ID"]
ws3.append(headers3)

for col, header in enumerate(headers3, 1):
    cell = ws3.cell(row=1, column=col)
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = header_alignment

for item in successful_inserts:
    ws3.append([item["es_id"], item["bbf_id"], item["bbf_service_id"]])

for col in ws3.columns:
    max_length = max(len(str(cell.value)) for cell in col)
    ws3.column_dimensions[col[0].column_letter].width = min(max_length + 2, 50)

ws3.freeze_panes = "A2"

# --- SHEET 4: Failed Inserts ---
ws4 = wb.create_sheet("Failed Inserts")
headers4 = ["ES OrderItem ID", "BBF Service ID", "Error Details"]
ws4.append(headers4)

for col, header in enumerate(headers4, 1):
    cell = ws4.cell(row=1, column=col)
    cell.font = Font(bold=True, size=12, color="FFFFFF")
    cell.fill = PatternFill(start_color="FF4444", end_color="FF4444", fill_type="solid")
    cell.alignment = header_alignment

for item in failed_inserts:
    ws4.append([item["es_id"], item["bbf_service_id"], str(item["errors"])])

for item in skipped_no_bbf_service:
    ws4.append([item["es_id"], "", item["reason"]])

for col in ws4.columns:
    col_cells = list(col)
    max_length = max(len(str(cell.value)) for cell in col_cells) if col_cells else 10
    ws4.column_dimensions[col[0].column_letter].width = min(max_length + 2, 70)

ws4.freeze_panes = "A2"

# Save workbook
wb.save(output_file)
print(f"\n✅ Excel output saved to: {output_file}")
print(f"   📊 Sheet 1: Migration Results ({len(all_results)} Service Charges)")
print(f"   📈 Sheet 2: Summary")
print(f"   🔗 Sheet 3: ID Mapping ({len(successful_inserts)} mappings)")
print(
    f"   ⚠️  Sheet 4: Failed Inserts ({len(failed_inserts) + len(skipped_no_bbf_service)} failures)"
)


CREATING EXCEL OUTPUT

✅ Excel output saved to: es_bbf_service_charge_migration_20260123_161715.xlsx
   📊 Sheet 1: Migration Results (17709 Service Charges)
   📈 Sheet 2: Summary
   🔗 Sheet 3: ID Mapping (17704 mappings)
   ⚠️  Sheet 4: Failed Inserts (5 failures)


In [9]:
# === FINAL SUMMARY ===

print("\n" + "=" * 80)
print("MIGRATION COMPLETE")
print("=" * 80)
print(f"ES OrderItem queried: {len(es_orderitems)}")
print(f"BBF Service_Charge__c inserted: {len(successful_inserts)}")
print(
    f"Success rate: {len(successful_inserts)/len(es_orderitems)*100:.1f}%"
    if len(es_orderitems) > 0
    else "N/A - No Service Charges processed"
)
print(f"\nExcel output: {output_file}")

if TEST_MODE:
    print(f"\n🔄 TEST MODE complete. Only migrated {TEST_LIMIT} Service Charges.")
    print(
        "   To migrate ALL Service Charges, set TEST_MODE = False in Cell 2 and re-run."
    )
else:
    print("\n✅ FULL MIGRATION complete!")
    print("   Service_Charge migration finished.")
    print("   Next: Migrate Off_Net__c (ES Off_Net__c → BBF Off_Net__c)")

if len(failed_inserts) > 0:
    print(f"\n⚠️  {len(failed_inserts)} Service Charges failed to insert")
    print("   Check 'Failed Inserts' sheet in Excel for details")

if len(skipped_no_bbf_service) > 0:
    print(
        f"\n⚠️  {len(skipped_no_bbf_service)} Service Charges skipped (no BBF Service - MASTER-DETAIL REQUIRED)"
    )
    print("   These OrderItems must have their parent Service migrated first")


MIGRATION COMPLETE
ES OrderItem queried: 17709
BBF Service_Charge__c inserted: 17704
Success rate: 100.0%

Excel output: es_bbf_service_charge_migration_20260123_161715.xlsx

✅ FULL MIGRATION complete!
   Service_Charge migration finished.
   Next: Migrate Off_Net__c (ES Off_Net__c → BBF Off_Net__c)

⚠️  5 Service Charges failed to insert
   Check 'Failed Inserts' sheet in Excel for details


---
## Day 1 Migration Summary

### Required Fields Set
| BBF Field | Value | Notes |
|-----------|-------|-------|
| Service__c | ES Order.BBF_New_Id__c | Master-Detail (BLOCKING) |
| Product_Simple__c | "ANNUAL" | PLACEHOLDER - needs enrichment |
| Service_Type_Charge__c | "Power" | PLACEHOLDER - needs enrichment |
| ES_Legacy_ID__c | ES OrderItem.Id | Tracking |

### Fields NOT Set (Why)
| Field | Reason |
|-------|--------|
| Name | Autonumber - auto-generated |
| OwnerId | Does not exist on Service_Charge__c |
| All Boolean fields | Default to False |

---
## Day 2+ Enrichment: Product Mapping

Once the business provides the ES Product → BBF Product mapping, run an enrichment process to update:
- `Product_Simple__c` - from placeholder to actual mapped value
- `Service_Type_Charge__c` - from placeholder to actual mapped value

### Enrichment Approach

**Option 1: Query by ES_Legacy_ID__c**
```python
# Query BBF Service Charges that have placeholders
bbf_charges = bbf_sf.query_all("""
    SELECT Id, ES_Legacy_ID__c, Product_Simple__c, Service_Type_Charge__c
    FROM Service_Charge__c
    WHERE ES_Legacy_ID__c != null
    AND Product_Simple__c = 'ANNUAL'
""")

# Query ES OrderItems with Product info
es_items = es_sf.query_all("""
    SELECT Id, Product2.Name, Product2.Family, Charge_Type__c
    FROM OrderItem
    WHERE Id IN ({ids_from_above})
""")

# Apply mapping and update
updates = []
for charge in bbf_charges:
    es_item = find_es_item(charge["ES_Legacy_ID__c"])
    mapped_product = PRODUCT_MAPPING.get(es_item["Product2"]["Name"], "Other")
    mapped_type = TYPE_MAPPING.get(es_item["Charge_Type__c"], "MRC")
    updates.append({
        "Id": charge["Id"],
        "Product_Simple__c": mapped_product,
        "Service_Type_Charge__c": mapped_type
    })

bbf_sf.bulk.Service_Charge__c.update(updates)
```

**Option 2: Separate Enrichment Notebook**
Create `enrichment/service_charge_product_mapping.ipynb` when mapping is ready.

### Mapping Template (for business to complete)
| ES Product2.Name | BBF Product_Simple__c | BBF Service_Type_Charge__c |
|------------------|----------------------|---------------------------|
| Product A | ? | ? |
| Product B | ? | ? |
| ... | ... | ... |

---
## Next Steps: Off_Net__c Migration

After Service_Charge migration is complete, migrate Off_Net__c.

### Prerequisites for Off_Net__c Migration
| Prerequisite | Status | Notes |
|--------------|--------|-------|
| Service__c | ✅ Complete | Service migration done |
| Service_Charge__c | 🔄 This notebook | Use ID Mapping sheet |

---
## Cleanup Apex (if needed)

### Delete Migrated Service Charges from BBF
```apex
List<Service_Charge__c> charges = [SELECT Id FROM Service_Charge__c WHERE ES_Legacy_ID__c != null];
System.debug('Found ' + charges.size() + ' migrated Service Charges');
delete charges;
```

### Remove BBF_New_Id__c from ES (to re-run migration)
```apex
List<OrderItem> items = [SELECT Id, BBF_New_Id__c 
                         FROM OrderItem 
                         WHERE BBF_New_Id__c != NULL];
System.debug('Found ' + items.size() + ' records to reset');
for (OrderItem item : items) {
    item.BBF_New_Id__c = NULL;
}
update items;
```